# Uncertainty calculation for model: SD1

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites, images and settings

In [4]:
#definet the color pallette
vibgYOR = ['330044', '220066', '1133cc', '33dd00', 'ffda21', 'ff6622', 'd10000']
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec");
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)
# generete the pixel area map
pixelArea = ee.Image.pixelArea().divide(10000) # to ha unit
# load the biome layer
biomeLayer = compositeImage.select("WWF_Biome")
biomeMask = biomeLayer.mask(biomeLayer.neq(98)).mask(biomeLayer.neq(99)).gt(0)
# load the mean maps for present and potential
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the biomass density layers
mergedAGB_PresentMean =  ee.Image("users/leonidmoore/ForestBiomass/RemoteSensingModel/ESA_CCI_AGB_Map_bias_corrected_1km_2010").unmask().multiply(carbonConcentration) 
mergedAGB_PotentialMean = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD1_Potential_density_Ensambled_Mean").unmask()
# define the standardized projection
stdProj = mergedAGB_PresentMean.projection()
# load the two forest cover layer for existing and potential forest
presentForestCover = compositeImage.select('PresentTreeCover').unmask()# make sure it's in  0-1 scale
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask() # make sure it's in  0-1 scale

# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialForestCover.gt(0)

## 2 Calculate the present and potential AGB

In [5]:
# check the difference of the two density maps
potentialHigher = mergedAGB_PotentialMean.multiply(pixelArea).subtract(mergedAGB_PresentMean.multiply(pixelArea)).gte(0)
potentialLower = mergedAGB_PotentialMean.multiply(pixelArea).subtract(mergedAGB_PresentMean.multiply(pixelArea)).lt(0)
# replace the lower potential value by present biomass density value
potentialAGB_Density = mergedAGB_PresentMean.multiply(potentialLower).add(mergedAGB_PotentialMean.multiply(potentialHigher))
presentAGB_Density = mergedAGB_PresentMean
# get the abs of present and potential AGB
presentAGB_Abs = presentAGB_Density.multiply(pixelArea).multiply(presentMask).divide(1000000000)
potentialAGB_Abs = potentialAGB_Density.multiply(pixelArea).multiply(potentialMask).divide(1000000000)

# presentAGB_Abs_Sum = presentAGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                  geometry = unboundedGeo,
#                                                  crs = 'EPSG:4326',
#                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                  maxPixels = 1e9)
# # print the estimation out
# print(colored('The present AGB:', 'blue', attrs=['bold']),presentAGB_Abs_Sum.getInfo())
# potentialAGB_Abs_Sum = potentialAGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                      geometry = unboundedGeo,
#                                                      crs = 'EPSG:4326',
#                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                      maxPixels = 1e9)
# # print the estimation out
# print(colored('The potential AGB:', 'blue', attrs=['bold']),potentialAGB_Abs_Sum.getInfo())

## 3 Calculate the Upper and Lower of present and potential AGB

In [6]:
# load the present prediction lower and upper layer
mergedPredictionInterval = ee.Image("users/leonidmoore/ForestBiomass/RemoteSensingModel/ESA_CCI_Biomass_Map_SD_1km_2010").unmask().multiply(carbonConcentration)
presentAGB_Lower1 = mergedAGB_PresentMean.subtract(mergedPredictionInterval)
presentAGB_Lower = presentAGB_Lower1.mask(presentAGB_Lower1.gte(0)).unmask()
presentAGB_Upper = mergedAGB_PresentMean.add(mergedPredictionInterval)

# get the upper and lower layer
mergedAGB_PotentialLower = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD1_Potential_density_Ensambled_Percentile").select(['lower']).unmask() 
mergedAGB_PotentialUpper = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_SD1_Potential_density_Ensambled_Percentile").select(['upper']).unmask() 

mergedAGB_PotentialLower1 = presentAGB_Lower.multiply(potentialLower).add(mergedAGB_PotentialLower.multiply(potentialHigher))
mergedAGB_PotentialUpper1 = presentAGB_Upper.multiply(potentialLower).add(mergedAGB_PotentialUpper.multiply(potentialHigher))

# define the masks to mask the present and potential lower maps
potentialAGB_Lower_Larger = mergedAGB_PotentialLower1.subtract(presentAGB_Lower).gte(0) # potential is larger than present mean
potentialAGB_Lower_Smaller = mergedAGB_PotentialLower1.subtract(presentAGB_Lower).lt(0)
# define the masks to mask the present and potential upper maps
potentialAGB_Upper_Larger = mergedAGB_PotentialUpper1.subtract(presentAGB_Upper).gte(0) # potential is larger than present upper
potentialAGB_Upper_Smaller = mergedAGB_PotentialUpper1.subtract(presentAGB_Upper).lt(0)

# replace the lower potential value by present biomass density value
potentialAGB_AdjLower = mergedAGB_PotentialLower.multiply(potentialAGB_Lower_Larger).add(presentAGB_Lower.multiply(potentialAGB_Lower_Smaller))
potentialAGB_AdjUpper = mergedAGB_PotentialUpper.multiply(potentialAGB_Upper_Larger).add(presentAGB_Upper.multiply(potentialAGB_Upper_Smaller))

# present lower and higher
presentAGB_Lower_Abs = presentAGB_Lower.multiply(pixelArea).multiply(presentMask).divide(1000000000)
presentAGB_Upper_Abs = presentAGB_Upper.multiply(pixelArea).multiply(presentMask).divide(1000000000)

# abs potential lower and higher
potentialAGB_Lower_Abs = potentialAGB_AdjLower.multiply(pixelArea).multiply(potentialMask).divide(1000000000)
potentialAGB_Upper_Abs = potentialAGB_AdjUpper.multiply(pixelArea).multiply(potentialMask).divide(1000000000)

In [7]:
# # Calculate the present AGB lower
# presentAGB_Lower_Sum = presentAGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                          geometry = unboundedGeo,
#                                                          crs = 'EPSG:4326',
#                                                          crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                          maxPixels = 1e9)

# # print the estimation out
# print(colored('The present AGB Lower:', 'blue', attrs=['bold']),presentAGB_Lower_Sum.getInfo())

# # Calculate the present AGB upper
# presentAGB_Upper_Sum = presentAGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                          geometry = unboundedGeo,
#                                                          crs = 'EPSG:4326',
#                                                          crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                          maxPixels = 1e9)

# # print the estimation out
# print(colored('The present AGB Upper:', 'blue', attrs=['bold']),presentAGB_Upper_Sum.getInfo())

# potentialAGB_Lower_Sum = potentialAGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                              geometry = unboundedGeo,
#                                                              crs = 'EPSG:4326',
#                                                              crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                              maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential AGB Lower:', 'blue', attrs=['bold']),potentialAGB_Lower_Sum.getInfo())

# potentialAGB_Upper_Sum = potentialAGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                              geometry = unboundedGeo,
#                                                              crs = 'EPSG:4326',
#                                                              crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                              maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential AGB Upper:', 'blue', attrs=['bold']),potentialAGB_Upper_Sum.getInfo())


## 4 Calculate the Upper and Lower of present and potential Root and TGB

In [8]:
# load the root shoot map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
rootShootRatioLower = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_percentile_Map").select('lower').unmask()
rootShootRatioUpper = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_percentile_Map").select('upper').unmask()
# 
presentRoot_Lower_Abs = presentAGB_Lower_Abs.multiply(rootShootRatioLower).mask(presentMask)
presentRoot_Upper_Abs = presentAGB_Upper_Abs.multiply(rootShootRatioUpper).mask(presentMask)

potentialRoot_Lower_Abs = potentialAGB_Lower_Abs.multiply(rootShootRatioLower).mask(potentialMask)
potentialRoot_Upper_Abs = potentialAGB_Upper_Abs.multiply(rootShootRatioUpper).mask(potentialMask)

presentRoot_Abs = presentAGB_Abs.multiply(rootShootRatio).mask(presentMask)
potentialRoot_Abs = potentialAGB_Abs.multiply(rootShootRatio).mask(potentialMask)

presentTGB_Abs = presentAGB_Abs.multiply(rootShootRatio).add(presentAGB_Abs)#.multiply(presentMask)
potentialTGB_Abs = potentialAGB_Abs.multiply(rootShootRatio).add(potentialAGB_Abs)#.multiply(potentialMask)

presentTGB  = presentAGB_Density.multiply(rootShootRatio.add(1))
# density 
presentRoot = presentAGB_Density.multiply(rootShootRatio)
presentRoot_Lower = presentAGB_Lower.multiply(rootShootRatioLower)
presentRoot_Upper = presentAGB_Upper.multiply(rootShootRatioLower)

potentialRoot_Lower = potentialAGB_AdjLower.multiply(rootShootRatioLower)
potentialRoot_Upper = potentialAGB_AdjUpper.multiply(rootShootRatioLower)

presentTGB_Lower = presentAGB_Lower.multiply(rootShootRatioLower).add(presentAGB_Lower)
presentTGB_Upper = presentAGB_Upper.multiply(rootShootRatioLower).add(presentAGB_Upper)

potentialTGB_Lower = potentialAGB_AdjLower.multiply(rootShootRatioLower).add(potentialAGB_AdjLower)
potentialTGB_Upper = potentialAGB_AdjUpper.multiply(rootShootRatioLower).add(potentialAGB_AdjUpper)

# presentTGB_Abs_Sum = presentTGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                  geometry = unboundedGeo,
#                                                  crs = 'EPSG:4326',
#                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                  maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB:', 'blue', attrs=['bold']),presentTGB_Abs_Sum.getInfo())

# potentialTGB_Abs_Sum = potentialTGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                      geometry = unboundedGeo,
#                                                      crs = 'EPSG:4326',
#                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                      maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB:', 'blue', attrs=['bold']),potentialTGB_Abs_Sum.getInfo())

# presentRoot_Abs_Sum = presentRoot_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                    geometry = unboundedGeo,
#                                                    crs = 'EPSG:4326',
#                                                    crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                    maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots:', 'blue', attrs=['bold']),presentRoot_Abs_Sum.getInfo())

# potentialRoot_Abs_Sum = potentialRoot_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots:', 'blue', attrs=['bold']),potentialRoot_Abs_Sum.getInfo())

# presentRoot_Lower_Sum = presentRoot_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots Lower:', 'blue', attrs=['bold']),presentRoot_Lower_Sum.getInfo())

# presentRoot_Upper_Sum = presentRoot_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots Upper:', 'blue', attrs=['bold']),presentRoot_Upper_Sum.getInfo())

# potentialRoot_Lower_Sum = potentialRoot_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots Lower:', 'blue', attrs=['bold']),potentialRoot_Lower_Sum.getInfo())
# potentialRoot_Upper_Sum = potentialRoot_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots Upper:', 'blue', attrs=['bold']),potentialRoot_Upper_Sum.getInfo())

In [9]:
presentTGB_Lower_Abs = presentAGB_Lower_Abs.multiply(rootShootRatioLower.add(1))
presentTGB_Upper_Abs = presentAGB_Upper_Abs.multiply(rootShootRatioUpper.add(1))

potentialTGB_Lower_Abs = potentialAGB_Lower_Abs.multiply(rootShootRatioLower.add(1))
potentialTGB_Upper_Abs = potentialAGB_Upper_Abs.multiply(rootShootRatioUpper.add(1))

# presentTGB_Lower_Sum = presentTGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB Lower:', 'blue', attrs=['bold']),presentTGB_Lower_Sum.getInfo())

# presentTGB_Upper_Sum = presentTGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB Upper:', 'blue', attrs=['bold']),presentTGB_Upper_Sum.getInfo())

# potentialTGB_Lower_Sum = potentialTGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB Lower:', 'blue', attrs=['bold']),potentialTGB_Lower_Sum.getInfo())
# potentialTGB_Upper_Sum = potentialTGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB Upper:', 'blue', attrs=['bold']),potentialTGB_Upper_Sum.getInfo())

## 5 Calculate the Upper and Lower of present and potential Root and PGB

In [10]:
# load the dead wood and litter layer
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()
deadWoodLitterRatioLower = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Lower_Map").unmask()
deadWoodLitterRatioUpper = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Upper_Map").unmask()

# calculate the present and potential PGB
presentPGB_Abs = presentTGB_Abs.multiply(deadWoodLitterRatio)
potentialPGB_Abs = potentialTGB_Abs.multiply(deadWoodLitterRatio)

# calculate the present and potential dead wood and litter
presentLitter_Abs = presentTGB_Abs.multiply(deadWoodLitterRatio.subtract(1))
potentialLitter_Abs = potentialTGB_Abs.multiply(deadWoodLitterRatio.subtract(1))

presentLitter = presentTGB.multiply(deadWoodLitterRatio.subtract(1))

# calculate the present Dead wood and litter
presentLitter_Lower_Abs = presentTGB_Lower_Abs.multiply(deadWoodLitterRatioLower.subtract(1))
presentLitter_Upper_Abs = presentTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper.subtract(1))
# calculate the potential dead wood and litter
potentialLitter_Lower_Abs = potentialTGB_Lower_Abs.multiply(deadWoodLitterRatioLower.subtract(1))
potentialLitter_Upper_Abs = potentialTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper.subtract(1))
# get the densities
presentPGB_Lower = presentTGB_Lower.multiply(deadWoodLitterRatioLower)
presentPGB_Upper = presentTGB_Upper.multiply(deadWoodLitterRatioUpper)

potentialPGB_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioLower)
potentialPGB_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioUpper)

presentLitter_Lower = presentTGB_Lower.multiply(deadWoodLitterRatioLower.subtract(1))
presentLitter_Upper = presentTGB_Upper.multiply(deadWoodLitterRatioUpper.subtract(1))

potentialLitter_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioLower.subtract(1))
potentialLitter_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioUpper.subtract(1))

# presentPGB_Abs_Sum = presentPGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB:', 'blue', attrs=['bold']),presentPGB_Abs_Sum.getInfo())

# potentialPGB_Abs_Sum = potentialPGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB:', 'blue', attrs=['bold']),potentialPGB_Abs_Sum.getInfo())

# presentLitter_Abs_Sum = presentLitter_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter:', 'blue', attrs=['bold']),presentLitter_Abs_Sum.getInfo())

# potentialLitter_Abs_Sum = potentialLitter_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Dead wood and litter:', 'blue', attrs=['bold']),potentialLitter_Abs_Sum.getInfo())

# presentLitter_Lower_Sum = presentLitter_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter Lower:', 'blue', attrs=['bold']),presentLitter_Lower_Sum.getInfo())

# presentLitter_Upper_Sum = presentLitter_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter Upper:', 'blue', attrs=['bold']),presentLitter_Upper_Sum.getInfo())

# potentialLitter_Lower_Sum = potentialLitter_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Dead wood and litter Lower:', 'blue', attrs=['bold']),potentialLitter_Lower_Sum.getInfo())

# potentialLitter_Upper_Sum = potentialLitter_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)
# # print the estimation out
# print(colored('The potential Dead wood and litter Upper:', 'blue', attrs=['bold']),potentialLitter_Upper_Sum.getInfo())


In [11]:
# calculate the present PGB Lower and Upper
presentPGB_Lower_Abs = presentTGB_Lower_Abs.multiply(deadWoodLitterRatioLower)
presentPGB_Upper_Abs = presentTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper)
# calculate the potential PGB Lower and Upper
potentialPGB_Lower_Abs = potentialTGB_Lower_Abs.multiply(deadWoodLitterRatioLower)
potentialPGB_Upper_Abs = potentialTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper)

presentPGB_D = presentAGB_Density.multiply(rootShootRatio.add(1)).multiply(deadWoodLitterRatio)
potentialPGB_D = potentialAGB_Density.multiply(rootShootRatio.add(1)).multiply(deadWoodLitterRatio)

# presentPGB_Lower_Sum = presentPGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB Lower:', 'blue', attrs=['bold']),presentPGB_Lower_Sum.getInfo())

# presentPGB_Upper_Sum = presentPGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB Upper:', 'blue', attrs=['bold']),presentPGB_Upper_Sum.getInfo())

# potentialPGB_Lower_Sum = potentialPGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB Lower:', 'blue', attrs=['bold']),potentialPGB_Lower_Sum.getInfo())

# potentialPGB_Upper_Sum = potentialPGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB Upper:', 'blue', attrs=['bold']),potentialPGB_Upper_Sum.getInfo())



## 6 Export the upper and lower images to Assets

In [12]:
# load the carbon density layers
SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# mask the diffrence layer
SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0))

# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialCoverAdjusted.gte(0.1)

# calculate the sum of the potential in soil with the consideration of forest cover
SandermannCarbonStockLoss = SandermannCarbonLoss.multiply(pixelArea).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

# add the soil into the PGB as the total potential
potentialTotal_Abs = potentialPGB_Abs.add(SandermannCarbonStockLoss)
# compose those bands into an image
lowerUpperImage = presentAGB_Lower_Abs.rename('preAGB_Lower').addBands(presentAGB_Upper_Abs.rename('preAGB_Upper')).addBands(potentialAGB_Lower_Abs.rename('potAGB_Lower')).addBands(potentialAGB_Upper_Abs.rename('potAGB_Upper')).addBands(presentRoot_Lower_Abs.rename('preRoot_Lower')).addBands(presentRoot_Upper_Abs.rename('preRoot_Upper')).addBands(potentialRoot_Lower_Abs.rename('potRoot_Lower')).addBands(potentialRoot_Upper_Abs.rename('potRoot_Upper')).addBands(presentLitter_Lower_Abs.rename('preLitter_Lower')).addBands(presentLitter_Upper_Abs.rename('preLitter_Upper')).addBands(potentialLitter_Lower_Abs.rename('potLitter_Lower')).addBands(potentialLitter_Upper_Abs.rename('potLitter_Upper')).addBands(potentialTotal_Abs.rename('PotentialTotal'))

In [13]:
exportUpperLower = ee.batch.Export.image.toAsset(image = lowerUpperImage,
                                               description = 'SD1_Upper_Lower_Uncertainty_maps_Export',
                                               assetId = 'users/leonidmoore/ForestBiomass/UncertaintyFigure/SD1_Lower_Upper_Map',
                                               region = unboundedGeo,
                                               crs = 'EPSG:4326',
                                               crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                               maxPixels = 1e13)
# start the export task
exportUpperLower.start()
# show the task status
exportUpperLower.status()

{'state': 'READY',
 'description': 'SD1_Upper_Lower_Uncertainty_maps_Export',
 'creation_timestamp_ms': 1690806798058,
 'update_timestamp_ms': 1690806798058,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'GNI7TRUQNXZ5C2VTYRY25C6B',
 'name': 'projects/earthengine-legacy/operations/GNI7TRUQNXZ5C2VTYRY25C6B'}

## 6 Calculate the Abs for different parts at biome level

In [16]:
# Stack the absolute biomass layers into an Image.
absImage = presentAGB_Abs.rename('PresentAGB').addBands(potentialAGB_Abs.rename('PotentialAGB')).addBands(presentRoot_Abs.rename('PresentRoot')).addBands(potentialRoot_Abs.rename('PotentialRoot')).addBands(presentTGB_Abs.rename('PresentTGB')).addBands(potentialTGB_Abs.rename('PotentialTGB')).addBands(presentLitter_Abs.rename('PresentLitter')).addBands(potentialLitter_Abs.rename('PotentialLitter')).addBands(presentPGB_Abs.rename('PresentPGB')).addBands(potentialPGB_Abs.rename('PotentialPGB'))

# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    perBiomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absImage.mask(perBiomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['PresentAGB','PotentialAGB','PresentRoot','PotentialRoot','PresentTGB','PotentialTGB','PresentLitter','PotentialLitter','PresentPGB','PotentialPGB']).round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/SD1_Abs_for_diff_parts_at_Biome_Level.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,PresentAGB,PotentialAGB,PresentRoot,PotentialRoot,PresentTGB,PotentialTGB,PresentLitter,PotentialLitter,PresentPGB,PotentialPGB
0,146.5,188.9,40.0,50.5,186.5,239.4,41.0,52.7,227.5,292.1
1,5.1,11.2,1.4,2.9,6.5,14.1,1.4,3.1,7.9,17.2
2,2.0,3.2,0.5,0.8,2.5,4.0,0.6,0.9,3.1,4.9
3,36.0,53.7,9.3,13.7,45.2,67.4,14.9,22.2,60.1,89.6
4,18.6,21.9,5.0,5.8,23.6,27.7,7.8,9.1,31.3,36.8
5,45.5,55.5,12.2,14.9,57.7,70.4,46.2,56.3,103.9,126.7
6,29.8,60.4,8.9,18.1,38.7,78.5,8.5,17.3,47.3,95.8
7,4.2,16.8,1.2,4.9,5.4,21.7,1.8,7.2,7.2,28.9
8,1.1,2.4,0.3,0.7,1.5,3.1,0.3,0.7,1.8,3.8
9,2.3,6.4,0.6,1.7,3.0,8.0,1.0,2.6,3.9,10.7


In [ ]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

## 7 Calculate the Abs for different parts at biome level

In [18]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = presentAGB_Lower_Abs.rename('preAGB_Lower').addBands(presentAGB_Upper_Abs.rename('preAGB_Upper')).addBands(potentialAGB_Lower_Abs.rename('potAGB_Lower')).addBands(potentialAGB_Upper_Abs.rename('potAGB_Upper')).addBands(presentRoot_Lower_Abs.rename('preRoot_Lower')).addBands(presentRoot_Upper_Abs.rename('preRoot_Upper')).addBands(potentialRoot_Lower_Abs.rename('potRoot_Lower')).addBands(potentialRoot_Upper_Abs.rename('potRoot_Upper')).addBands(presentLitter_Lower_Abs.rename('preLitter_Lower')).addBands(presentLitter_Upper_Abs.rename('preLitter_Upper')).addBands(potentialLitter_Lower_Abs.rename('potLitter_Lower')).addBands(potentialLitter_Upper_Abs.rename('potLitter_Upper'))
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    perBiomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(perBiomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['preAGB_Lower','preAGB_Upper','potAGB_Lower','potAGB_Upper','preRoot_Lower','preRoot_Upper','potRoot_Lower','potRoot_Upper','preLitter_Lower','preLitter_Upper','potLitter_Lower','potLitter_Upper']).round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/SD1_Uncertainty_for_diff_parts_at_Biome_Level.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,preAGB_Lower,preAGB_Upper,potAGB_Lower,potAGB_Upper,preRoot_Lower,preRoot_Upper,potRoot_Lower,potRoot_Upper,preLitter_Lower,preLitter_Upper,potLitter_Lower,potLitter_Upper
0,105.2,188.1,164.9,205.0,22.5,62.8,34.8,66.7,19.2,75.3,30.0,81.5
1,3.6,6.8,9.2,12.9,0.8,2.1,1.9,3.9,0.7,2.7,1.7,5.0
2,1.5,2.6,2.7,3.7,0.3,0.7,0.6,1.1,0.3,1.0,0.5,1.4
3,24.4,48.1,43.5,59.2,5.6,14.0,9.9,17.2,9.0,23.0,16.0,28.3
4,13.0,24.3,15.6,21.9,3.0,7.4,3.6,6.7,4.8,11.7,5.8,10.6
5,32.6,58.6,43.8,57.6,8.0,17.5,10.9,17.3,27.5,71.4,37.2,70.4
6,20.5,39.7,49.1,69.6,5.2,13.7,12.5,24.3,3.9,16.0,9.2,28.2
7,3.1,5.5,13.6,19.8,0.8,1.8,3.4,6.7,1.2,2.7,5.1,9.8
8,0.9,1.4,2.0,2.7,0.2,0.5,0.5,0.9,0.2,0.6,0.4,1.1
9,1.6,3.1,5.0,7.5,0.4,1.0,1.1,2.3,0.6,1.5,1.8,3.6


In [ ]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.